## Required Libraries

In [4]:
%pip install tensorboard

  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   -- ------------------------------------- 0.3/4.3 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.3 MB 4.0 MB/s eta 0:00:01
   --------- ------------------------------ 1.0/4.3 MB 1.7 MB/s eta 0:00:02
   ------------ --------------------------- 1.3/4.3 MB 1.8 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.3 MB 1.6 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.3 MB 1.6 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.3 MB 1.6 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.3 MB 1.6 MB/s eta 0:00:02
   -------------- ------------------------- 1.6/4.3 MB 1.6


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Dataset PreProcessing

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer

# Load data - FIXED splitting method
with open(r"C:\Users\adars\OneDrive\Desktop\Dataset.txt", "r", encoding="utf-8") as f:
    # Try both single and double newlines as separators
    content = f.read()
    tweets = [p.strip() for p in content.splitlines() if p.strip()]  # Split by any newline
    
    # Verify we got multiple tweets
    print(f"Loaded {len(tweets)} tweets")  # Debug output

# Ensure we have enough data
if len(tweets) < 2:
    raise ValueError(f"Only {len(tweets)} tweets found - need at least 2 for train/test split")

dataset = Dataset.from_dict({"text": tweets})

# Adjusted splitting - ensure minimum 1 sample in each set
test_size = max(1, int(0.1 * len(tweets)))  # At least 1 sample for test
dataset = dataset.train_test_split(
    test_size=test_size,
    train_size=len(tweets)-test_size,  # Explicit sizes
    seed=42
)

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=64  # Reduced for tweets
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\Users\adars\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded 158 tweets


Map: 100%|██████████| 15/15 [00:00<00:00, 3690.00 examples/s]


# Fine Tuning GPT2 Model

In [11]:
# Install packages without version constraints
!pip install torch transformers datasets

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import Dataset

# 1. Load and prepare dataset
with open("Dataset.txt", "r", encoding="utf-8") as f:
    tweets = [line.strip() for line in f.readlines() if line.strip()]

print(f"Loaded {len(tweets)} tweets")
if len(tweets) < 10:
    raise ValueError(f"Insufficient data: only {len(tweets)} tweets found. Need at least 10.")

dataset = Dataset.from_dict({"text": tweets})
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

# 2. Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Remove return_tensors="pt" for compatibility with datasets.map
    tokens = tokenizer(
        examples["text"],
        truncation=True,
        max_length=64,
        padding="max_length"
    )
    # Add labels for causal LM: labels = input_ids
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_train = split_dataset["train"].map(tokenize_function, batched=True)
tokenized_eval = split_dataset["test"].map(tokenize_function, batched=True)

# 3. Initialize model
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.config.pad_token_id = tokenizer.pad_token_id

# 4. Configure training arguments (universal parameters)
base_args = {
    "output_dir": "./results",
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "num_train_epochs": 50,
    "logging_steps": 100,
    "save_total_limit": 2,
    "learning_rate": 5e-5,
    "weight_decay": 0.01,
    "gradient_accumulation_steps": 2,
}

# Add version-specific parameters dynamically
try:
    training_args = TrainingArguments(
        **base_args,
        evaluation_strategy="steps",
        eval_steps=500,
        save_strategy="steps",
        save_steps=500,
        fp16=torch.cuda.is_available(),
    )
except TypeError:
    try:
        training_args = TrainingArguments(
            **base_args,
            evaluate_during_training=True,
            save_steps=500,
            fp16=torch.cuda.is_available(),
        )
    except TypeError:
        training_args = TrainingArguments(
            **base_args,
            fp16=torch.cuda.is_available(),
        )

# 5. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
)

# 6. Train the model
print("Starting training...")
trainer.train()

# 7. Save model
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

# 8. Generation function
def generate_tweet(prompt, max_length=50):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test generation
print("\nGenerated Tweet Examples:")
print(generate_tweet("I'm not saying I'm a good cook, but"))
print(generate_tweet("My dog is so smart,"))


Loaded 158 tweets



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\adars\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip
Map: 100%|██████████| 32/32 [00:00<00:00, 6721.98 examples/s]


Starting training...


c:\Users\adars\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,0.714200
200,0.189800
300,0.125900
400,0.107400
500,0.098000
600,0.094300
700,0.088500
800,0.088600


c:\Users\adars\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Generated Tweet Examples:
I'm not saying I'm a good cook, but my fire extinguisher has never been used. As a paperweight.
My dog is so smart, he has no limits.
